In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
#os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/kaggle"

In [3]:
#changing the working directory
#%cd /content/gdrive/My Drive/kaggle

In [4]:
#!kaggle datasets download -d mlg-ulb/creditcardfraud

In [5]:
#unzipping the zip files and deleting the zip files
#!unzip \*.zip  && rm *.zip

In [6]:
!pwd

/content


In [7]:
#Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [8]:
credit = pd.read_csv('/content/gdrive/My Drive/kaggle/creditcard.csv')
credit.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [9]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [10]:
credit.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [11]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [12]:
# selecting the 19 most important features according to our EDA 
X = credit[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 
        'V12', 'V13', 'V14', 'V16', 'V17', 'V21', 'V23', 'V27']]

# scaling the features
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)

y = credit['Class'] # selecting the target variable

In [13]:
# computing the class weight

val_count = credit['Class'].value_counts()
weights = dict(1 / val_count)
weights

{0: 3.51722561243691e-06, 1: 0.0020325203252032522}

In [14]:
# adminstrative cost
admin_cost = 2.5

In [15]:
# defining a function to calculate cost savings
def cost_saving(ytrue, ypred, amount):
    fp = np.sum((ytrue == 0) & (ypred == 1))
    cost = np.sum(fp*admin_cost) + np.sum((amount[(ytrue == 1) & (ypred == 0)]))
    max_cost = np.sum((amount[(ytrue == 1)]))
    savings = 1 - (cost/max_cost)
    
    return savings

In [16]:
cv = StratifiedKFold(n_splits=4, random_state=1, shuffle=True)

In [17]:
# defining a function to calculate cost saving per fold (splits) of our cv
def cost_saving_per_split(scores, x, y, cv_object):
    results = []
    for i, (_, test_ind) in zip(range(cv_object.n_splits), cv_object.split(x, y)):
        ypred = scores['estimator'][i].predict(x[test_ind])
        ytrue = y[test_ind]
        amount = credit['Amount'].values[test_ind]
        results.append(cost_saving(ytrue, ypred, amount))
        
    return results

In [67]:
# defining a function to return a dataframe of metrics results for each fold in our cv
def get_metric_scores(scores, x, y=y, cv_object=cv):
    ind = ['split_'+str(n) for n in range(1, cv_object.n_splits+1)]
    
    scores_credit = pd.DataFrame(index=ind)
    
    scores_credit['f1_score'] = scores['test_f1']
    scores_credit['auc_pr'] = scores['test_average_precision']
    scores_credit['cost_savings'] = cost_saving_per_split(scores, x, y, cv_object)

    return scores_credit

**Fraud Sensitive model (Not cost sensitive)**

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

#GradientBoosting = GradientBoostingClassifier()

In [ ]:
fraud_sensitive_model = GradientBoostingClassifier()
fraud_sensitive_scores = cross_validate(fraud_sensitive_model, scaled_X, y, 
                                        scoring=['f1', 'average_precision'], 
                                        cv=cv, n_jobs=4, return_estimator=True)

In [21]:
fraud_sensitive_results = get_metric_scores(fraud_sensitive_scores, scaled_X)
fraud_sensitive_results

,f1_score,auc_pr,cost_savings
split_1,0.463277,0.371886,0.495092
split_2,0.660194,0.418952,0.537342
split_3,0.767123,0.631065,0.678380
split_4,0.697674,0.514382,0.592574


In [22]:
fraud_sensitive_results.mean()

f1_score        0.647067
auc_pr          0.484071
cost_savings    0.575847
dtype: float64

**Cost Sensitive model**

In [23]:
sample_weights = np.array([credit['Amount'][ind] if fraud else admin_cost for ind, fraud in enumerate(y)])

In [24]:
cost_sensitive_model = GradientBoostingClassifier()
cost_sensitive_scores = cross_validate(cost_sensitive_model, scaled_X, y, 
                                       scoring=['f1', 'average_precision'], 
                                       cv=cv, n_jobs=4, return_estimator=True, 
                                       fit_params={'sample_weight': sample_weights})

In [25]:
cost_sensitive_results = get_metric_scores(cost_sensitive_scores, scaled_X)
cost_sensitive_results

,f1_score,auc_pr,cost_savings
split_1,0.745098,0.685282,0.723597
split_2,0.782609,0.754540,0.751653
split_3,0.808163,0.754870,0.813925
split_4,0.804979,0.746105,0.735220


In [26]:
cost_sensitive_results.mean()

f1_score        0.785212
auc_pr          0.735199
cost_savings    0.756099
dtype: float64

**Cost Sensitive Model - SMOTE**

In [53]:
# selecting the 19 most important features according to our EDA 
X_new = credit[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 
        'V12', 'V13', 'V14', 'V16', 'V17', 'V21', 'V23', 'V27', 'Amount']]



In [55]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_oversampled, y_oversampled = smote.fit_sample(X_new, y)

In [56]:
X_oversampled = pd.DataFrame(X_oversampled, columns=X_new.columns)
X_oversampled

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V16,V17,V21,V23,V27,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,-0.470401,0.207971,-0.018307,-0.110474,0.133558,149.620000
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.463917,-0.114805,-0.225775,0.101288,-0.008983,2.690000
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,-2.890083,1.109969,0.247998,0.909412,-0.055353,378.660000
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-1.059647,-0.684093,-0.108300,-0.190321,0.062723,123.500000
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,-0.451449,-0.237033,-0.009431,-0.137458,0.219422,69.990000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568625,-3.155616,1.767081,-3.398502,0.989428,-1.510791,-1.278360,-2.724025,0.724164,-1.656033,-3.849590,3.283952,-3.922658,0.952560,-4.893009,-4.781659,-9.324699,1.119009,-0.614580,-0.232642,84.260671
568626,-4.608251,3.662118,-5.293763,3.881100,-3.258690,-0.918828,-5.710748,0.833389,-2.459734,-5.728383,3.528937,-6.489639,-1.009931,-7.057308,-6.771309,-8.590487,2.692423,0.364411,-0.179450,1.000000
568627,1.172874,2.876345,-4.096471,4.769603,3.070751,-1.892667,1.738560,-0.425809,-2.239678,-2.115261,1.890486,-2.819744,-1.034476,-7.004710,2.412547,6.496868,-0.213065,-0.552313,0.006373,1.000000
568628,-11.384120,7.075869,-13.879024,8.660994,-10.843674,-2.356901,-13.033113,5.447322,-7.893342,-12.830543,6.196877,-12.400593,1.080001,-11.196340,-11.478495,-21.210056,1.646852,-1.327453,-0.215869,353.811171


In [57]:
X_new_features = X_oversampled[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 
        'V12', 'V13', 'V14', 'V16', 'V17', 'V21', 'V23', 'V27']]

X_new_features.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V16,V17,V21,V23,V27
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,-0.470401,0.207971,-0.018307,-0.110474,0.133558
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.463917,-0.114805,-0.225775,0.101288,-0.008983
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,-2.890083,1.109969,0.247998,0.909412,-0.055353
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-1.059647,-0.684093,-0.108300,-0.190321,0.062723
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,-0.451449,-0.237033,-0.009431,-0.137458,0.219422


In [60]:
y_oversampled.shape

(568630,)

In [61]:
sample_weight = np.array([X_oversampled['Amount'][ind] if fraud else admin_cost for ind, fraud in enumerate(y_oversampled)])

In [59]:
# scaling the features
scaler = StandardScaler()
scaled_X_new = scaler.fit_transform(X_new_features)

In [63]:
cost_sensitive_smote = GradientBoostingClassifier()
smote_scores = cross_validate(cost_sensitive_smote, scaled_X_new, y_oversampled, 
                                       scoring=['f1', 'average_precision'], 
                                       cv=cv, n_jobs=4, return_estimator=True, 
                                       fit_params={'sample_weight': sample_weight})

In [73]:
smote_results = get_metric_scores(smote_scores, scaled_X)
smote_results

,f1_score,auc_pr,cost_savings
split_1,0.963101,0.994522,-3.172615
split_2,0.961648,0.994510,-6.322049
split_3,0.962278,0.994502,-4.743623
split_4,0.962426,0.994510,-5.091978


In [74]:
smote_results.mean()

f1_score        0.962363
auc_pr          0.994511
cost_savings   -4.832566
dtype: float64

**Cost sensitive - Undersampling**

In [75]:
from imblearn.under_sampling import RandomUnderSampler

undersample = RandomUnderSampler()

X_undersampled, y_undersampled = undersample.fit_resample(X_new, y)

In [76]:
X_undersampled = pd.DataFrame(X_undersampled, columns=X_new.columns)
X_undersampled

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V16,V17,V21,V23,V27,Amount
0,-0.226609,0.961791,0.242349,-0.731235,0.335654,-0.885546,0.863096,-0.027474,-0.046959,0.081210,0.795820,0.610708,-0.572068,0.339361,0.077092,-0.606856,-0.239714,0.089724,0.347581,0.89
1,1.313611,-1.477745,0.862860,-1.229934,-2.138836,-0.723148,-1.316068,-0.042157,-1.333068,1.395608,-0.722183,-1.544768,-0.678814,-0.279068,0.019309,0.428771,-0.158245,0.044272,0.034713,101.30
2,0.124238,1.030743,-0.516255,-0.758945,1.252213,-0.207739,0.895305,0.041307,-0.401554,-0.694262,0.868079,0.887560,0.762354,-0.992397,0.537845,-0.004218,-0.300214,0.030397,0.227124,0.89
3,-1.872970,2.474065,0.860039,0.538976,-0.238178,0.341203,-0.663132,-2.483350,-0.645788,0.690268,1.523021,1.118067,0.890349,-0.182752,0.051538,0.342739,-1.604163,0.069157,0.694295,5.94
4,2.164319,-0.395081,-2.055745,0.455904,0.464475,-0.285354,0.137421,-0.260968,-0.717682,1.071328,-1.962492,-0.595524,-0.479333,0.458278,-1.977383,-0.113491,-0.360959,0.020789,-0.002282,26.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,-5.587794,2.115795,-5.417424,-1.235123,-6.665177,-2.897825,-4.570529,0.778584,0.639419,0.292680,390.00
980,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,-3.232153,2.858466,-3.096915,-0.792532,-5.210141,-2.155297,-3.267116,0.370612,-0.145640,0.389152,0.76
981,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,-3.463891,1.794969,-2.775022,-0.418950,-4.057162,-1.603015,-5.035326,0.751826,0.190944,0.385107,77.89
982,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,-5.245984,1.933520,-5.030465,-1.127455,-6.416628,-2.549498,-4.614717,0.583276,-0.456108,0.884876,245.00


In [77]:
X_new_feat = X_undersampled[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 
        'V12', 'V13', 'V14', 'V16', 'V17', 'V21', 'V23', 'V27']]

X_new_feat.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V16,V17,V21,V23,V27
0,-0.226609,0.961791,0.242349,-0.731235,0.335654,-0.885546,0.863096,-0.027474,-0.046959,0.081210,0.795820,0.610708,-0.572068,0.339361,0.077092,-0.606856,-0.239714,0.089724,0.347581
1,1.313611,-1.477745,0.862860,-1.229934,-2.138836,-0.723148,-1.316068,-0.042157,-1.333068,1.395608,-0.722183,-1.544768,-0.678814,-0.279068,0.019309,0.428771,-0.158245,0.044272,0.034713
2,0.124238,1.030743,-0.516255,-0.758945,1.252213,-0.207739,0.895305,0.041307,-0.401554,-0.694262,0.868079,0.887560,0.762354,-0.992397,0.537845,-0.004218,-0.300214,0.030397,0.227124
3,-1.872970,2.474065,0.860039,0.538976,-0.238178,0.341203,-0.663132,-2.483350,-0.645788,0.690268,1.523021,1.118067,0.890349,-0.182752,0.051538,0.342739,-1.604163,0.069157,0.694295
4,2.164319,-0.395081,-2.055745,0.455904,0.464475,-0.285354,0.137421,-0.260968,-0.717682,1.071328,-1.962492,-0.595524,-0.479333,0.458278,-1.977383,-0.113491,-0.360959,0.020789,-0.002282


In [78]:
undersample_weight = np.array([X_undersampled['Amount'][ind] if fraud else admin_cost for ind, fraud in enumerate(y_undersampled)])

In [79]:
# scaling the features
scaler = StandardScaler()
scaled_new_X = scaler.fit_transform(X_new_feat)

In [80]:
cost_sensitive_under = GradientBoostingClassifier()
under_scores = cross_validate(cost_sensitive_under, scaled_new_X, y_undersampled, 
                                       scoring=['f1', 'average_precision'], 
                                       cv=cv, n_jobs=4, return_estimator=True, 
                                       fit_params={'sample_weight': undersample_weight})

In [82]:
undersample_results = get_metric_scores(under_scores, scaled_X)
undersample_results

,f1_score,auc_pr,cost_savings
split_1,0.918288,0.983163,-3.611872
split_2,0.911290,0.969172,-8.559496
split_3,0.896000,0.971427,-6.476781
split_4,0.920000,0.980632,-6.116415


In [83]:
undersample_results.mean()

f1_score        0.911395
auc_pr          0.976098
cost_savings   -6.191141
dtype: float64